# Assignment 4
### Last updated: April 25, 2022

### Name: your_name

### Uniqname: your_uniqname

## Instructions

Please turn in:
1. A Jupyter Notebook file. This file should show all of the required work, including code, results, visualizations (if any), and necessary comments to your code. Irrelevant code and results should be deleted prior to submission.

2. An html file showing the preview of the Notebook. To create this file, select File -> Download as > HTML. 

### Before submitting, please select Kernel -> Restart & Run All.

In [1]:
import re
import time
import pickle

from networkx.drawing.nx_pydot import graphviz_layout
from networkx.algorithms import community
from networkx.algorithms.community import modularity
import networkx as nx

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

## <font color='red'>Important</font>

<font color='red' size=2>Please **AVOID** using `community` and `modularity` as your variable names. These are imported as preserved names for networkx submodules. Changing their representations would result in autograder failuers.</font>

In [2]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

## Part 1. Wikipedia Network with Communities

## Data description

In this assignment, we are going to analyze the community structure of a network. We will use a Wikipedia based [Map of Science](https://figshare.com/articles/A_Wikipedia_Based_Map_of_Science/11638932) network for our exploration. In this network, each node represents a Wikipedia page in a domain of science, such as natural science or social science. An edge exists between two nodes if the cosine similarity of their page contents reaches a pre-defined threshold.

In [3]:
G = nx.read_gml('assets/MapOfScience.gml', label='id')

Each node in the graph contains the attributes:
- "name:" the title of the article 
- "Class" the science domain 
- "WikipediaUrl:" the Wikipedia URL 

Let's look at some examples: 

In [4]:
list(G.nodes(data=True))[0:5]

[(0,
  {'label': '0',
   'name': 'Accounting',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Accounting'}),
 (1,
  {'label': '1',
   'name': 'Aerospace engineering',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Aerospace_engineering'}),
 (2,
  {'label': '2',
   'name': 'Agricultural engineering',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Agricultural_engineering'}),
 (3,
  {'label': '3',
   'name': 'Agricultural science',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Agricultural_science'}),
 (4,
  {'label': '4',
   'name': 'Agronomy',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Agronomy'})]

The edges contain the cosine similarity of the text of the two articles：

In [5]:
list(G.edges(data=True))[0:5]

[(0, 21, {'CosineSimilarity': 0.369447477753246}),
 (0, 50, {'CosineSimilarity': 0.395432741205435}),
 (0, 70, {'CosineSimilarity': 0.388758063740006}),
 (0, 88, {'CosineSimilarity': 0.371542879166867}),
 (0, 516, {'CosineSimilarity': 0.365688238862527})]

### Q1. (1 point, Autograded) Let's extract the largest connected component from the above graph. In the following questions, we will focus our analysis on this sub-graph. How many nodes are there in this new network?

In [6]:
# Extract the largest connected component of the original dataset
G = G.subgraph(max(nx.connected_components(G), key=len))
N = None  # stores the number of nodes in this graph

# YOUR CODE HERE
N = len(G.nodes())
# raise NotImplementedError()

In [7]:
#hidden tests for Question 1 are within this cell

### Q2. (2 points, Autograded) If we think of science domains as communities, how many communities are there in the network and what are their Classes?

In [8]:
list_of_communities = set() # set of unique community labels
N = None           # number of communities in total

# YOUR CODE HERE
# Create a set of unique community labels
list_of_communities = set([data['Class'] for node, data in G.nodes(data=True)])

# Find the total number of communities
N = len(list_of_communities)
# raise NotImplementedError()

In [9]:
#hidden tests for Question 2 are within this cell

### Q3. (4 points, Autograded) How many nodes does each community have?

Hint: you may want to use the [`Counter`](https://docs.python.org/2/library/collections.html#counter-objects) object for this question.

In [10]:
from collections import Counter
dict_num_community = None   # a Counter object with the format {community_name: number_of_nodes}

# YOUR CODE HERE
community_list = [data['Class'] for node, data in G.nodes(data=True)]
# Use Counter to count the number of nodes in each community
dict_num_community = Counter(community_list)
# raise NotImplementedError()

In [11]:
#hidden tests for Question 3 are within this cell

## Part 2. Measures of Partition Quality

We discussed 5 ways to measure the quality of a partition: modularity, coverage, performance, separability, and density. 

Modularity, coverage, and performance can be measured using `networkx` functions in the `algorithms.community` module. You can check all the functions provided by a module with the built-in `dir()` function:

```python
from networkx.algorithms import community
dir(community)
>>> ...
 'coverage',
 'modularity',
 'performance',
```

Descriptions of the three measures are provided in the source code,

1. `networkx.algorithms.community.modularity(G, communities, weight='weight')`

2. [`networkx.algorithms.community.quality.coverage(G, partition)`](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.coverage.html#networkx.algorithms.community.quality.coverage)

3. [`networkx.algorithms.community.quality.performance(G, partition)`](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.performance.html#networkx.algorithms.community.quality.performance)





For separability and density, you will implement your own functions. 

Since separability and density first apply a measure to each community and then takes the average over all communities, we provide a helper function `avg_measure(G, communities, measure)` that computes the average value for a given measure over all communities in a graph:

In [12]:
def avg_measure(G, communities, measure):
    """
    Calculate the average value of a given measure across communities in a graph.
    
    Args:
        G - nx graph object; a graph to operate upon.
        communities - list; a collection of sets of nodes that each comprise a community.
        measure - function; calculates a measure for a community in a graph.
    
    Returns:
        (unnamed) - float; the calculated average measure.
    """
    sum_ = 0
    for comm in communities:
        sum_ += measure(G, comm)
    return sum_ / len(communities)

### Q4. (10 points, Autograded) Let's begin implementing the function to measure the separability for a single community. That is, measure the ratio of intra-community to inter-community edges. 

If there are 0 inter-community edges, the function should assume that the actual number is 1, making the separability value equal to the number of intra-community edges. 

Hint: `G.edges(community)`, where `community` is a set of nodes, returns the edges that are incident to at least one node in `community`. That is, it returns the edges that have at least one endpoint in `community`. 

In [13]:
def separability_one_community(G, community):
    """
    Calculate the separability of a community by finding the ratio of
    intra-community edges to inter-community edges.
    
    Args:
        G - nx graph object; a graph to operate upon.
        community - set; a collection of nodes that comprise a community.
    
    Returns:
        result - float; the separability in a given community.
    """
    # Create a subgraph for the community
    subgraph = G.subgraph(community)
    # Count the intra-community edges
    intra_edges = subgraph.number_of_edges()
    # Count the inter-community edges
    inter_edges = len(G.edges(community)) - intra_edges
    # Calculate separability
    result = intra_edges / (inter_edges if inter_edges > 0 else 1)
#     result = None # assign separability to it and return
    return result
    

# YOUR CODE HERE
# raise NotImplementedError()

In [14]:
#hidden tests for Question 4 are within this cell

Now we can simply use `avg_measure(G, communities, separability_one_community)` to measure the separability of a partition. 

### Q5. (10 points, Autograded) Let's now implement the function to measure the density of a single community. That is, the fraction of intra-community edges out of all possible edges. 

In [15]:
def density_one_community(G, community):
    """
    Calculate the density of a community by finding the fraction of
    intra-community edges out of all possible edges.
    
    Args:
        G - nx graph object; a graph to operate upon.
        community - set; a collection of nodes that comprise a community.
    
    Returns:
        result - float; the density of a given community.
    """
    
    result = None # assign density to it and return    
    
    if len(community) == 1:  # If the community has only one node, just return 1 
        return 1
    # YOUR CODE HERE
    # Create a subgraph for the community
    subgraph = G.subgraph(community)
    # Count the intra-community edges
    intra_edges = subgraph.number_of_edges()
    # Calculate the total possible edges
    total_possible_edges = len(community)*(len(community)-1) / 2
    # Calculate density
    result = intra_edges / total_possible_edges
#     raise NotImplementedError()
    return result

In [16]:
#hidden tests for Question 5 are within this cell

Now we can simply use `avg_measure(G, communities, density_one_community)` to measure the density of a partition. 

### Q6. (5 points, Autograded) What is the modularity, coverage, performance, density, and separability of this network using the science domain as a partition?

In [17]:
mod = None   # modularity
cov = None   # coverage
perf = None  # performance
den = None # density
sep = None # separability

# YOUR CODE HERE
from collections import defaultdict
from networkx.algorithms.community import quality
# Create a dictionary where keys are classes (science domains) and values are sets of nodes in that class
class_dict = defaultdict(set)

for node, data in G.nodes(data=True):
    class_dict[data['Class']].add(node)
# Convert the values in the dictionary to a list to get the list of communities
communities = list(class_dict.values())
# Calculate modularity
mod = quality.modularity(G, communities)

# Calculate coverage
cov = quality.coverage(G, communities)

# Calculate performance
perf = quality.performance(G, communities)

# Calculate density
den = avg_measure(G, communities, density_one_community)

# Calculate separability
sep = avg_measure(G, communities, separability_one_community)

# raise NotImplementedError()

In [18]:
result = {}
num_community = len(communities)
modularity = mod
coverage = cov
performance = perf
density = den
separability = sep
result['science_domain_partitions'] = [num_community,modularity,coverage,performance,density,separability]
science_domain = pd.DataFrame(result).T
science_domain.columns = ['num_community','modularity','coverage','performance','density','separability']
science_domain

,num_community,modularity,coverage,performance,density,separability
science_domain_partitions,4.0,0.39848,0.715206,0.742542,0.069375,1.140328


In [19]:
#hidden tests for Question 6 are within this cell

## Part 3. Community Detection Algorithms

Now let's apply community detection algorithms to the Wikipedia graph. For this part, we will ignore the science domains since we are trying to find communities purely based on the structure of the network. 

We will begin with the Girvan-Newman algorithm and pick the partition that results in the largest modularity, as described in the lecture. Since computing this partition takes a long time, we have precomputed the communities in the notebook `Girvan-Newman.ipynb` (stored in the resources folder) and exported the partition to a pickle file. The pickle file is stored as `assets/answer/max_mod_community`. Note that you do not need to run the notebook `Girvan-Newman.ipynb` since we have already done it for you. We only provide it to you as a reference.

### Q7. (1 point, Autograded) Load the partition from `assets/answer/max_mod_community`. How many communities are there in this partition?

**Hint**:

The partition is stored as a pickle file, which can be imported as the following example:
```python
with open("my_pickle_file_name", 'rb') as f:
    my_data = pickle.load(f)

```

In [20]:
num_max_mod_communities = None  # number of communities in the partition with the largest modularity value

# YOUR CODE HERE
with open("assets/answer/max_mod_community", 'rb') as f:
    max_mod_communities = pickle.load(f)
# Calculate the number of communities
num_max_mod_communities = len(max_mod_communities)
# raise NotImplementedError()

In [21]:
#hidden tests for Question 7 are within this cell

### Q8. (5 points, Autograded) What is the modularity, coverage, performance, density, and separability of this partition?

In [22]:
mod = None   # modularity
cov = None   # coverage
perf = None  # performance
den = None # density
sep = None # separability


# YOUR CODE HERE
# Calculate modularity, coverage, performance, density, separability
mod = quality.modularity(G, max_mod_communities)
cov = quality.coverage(G, max_mod_communities)
perf = quality.performance(G, max_mod_communities)
den = avg_measure(G, max_mod_communities, density_one_community)
sep = avg_measure(G, max_mod_communities, separability_one_community)
# raise NotImplementedError()

In [23]:
#hidden tests for Question 8 are within this cell

### Q9. (12 points, Autograded) Find a partition of the network with the [label propagation algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.label_propagation.label_propagation_communities.html) and compute the number of communities in the partition and its modularity, coverage, performance, denisty, and separability. This function uses semi-synchronous updating.

In [24]:
num_community = None # number of communities in the partition
mod = None   # modularity
cov = None   # coverage
perf = None  # performance
den = None # density
sep = None # separability

from networkx.algorithms import community

# Find a partition with the label propagation algorithm
label_prop_communities = list(community.label_propagation_communities(G))
num_community = len(label_prop_communities)
# Calculate modularity, coverage, performance, density, separability
mod = quality.modularity(G, label_prop_communities)
cov = quality.coverage(G, label_prop_communities)
perf = quality.performance(G, label_prop_communities)
den = avg_measure(G, label_prop_communities, density_one_community)
sep = avg_measure(G, label_prop_communities, separability_one_community)

# YOUR CODE HERE
# raise NotImplementedError()

In [25]:
#hidden tests for Question 9 are within this cell

The [Clauset-Newman-Moore greedy modularity maximization algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.modularity_max.greedy_modularity_communities.html#networkx-algorithms-community-modularity-max-greedy-modularity-communities) implements an Agglomerative Hierarchical Clustering procedure to find a partition with high modularity. 

Note: the function `greedy_modularity_communities` returns a list of `Frozensets`, where each `Frozenset` is simply an immutable Python `set` object (i.e. the elements cannot be modified).



### Q10. (12 points, Autograded) Using Clauset-Newman-Moore greedy modularity maximization, find a partition of the network and compute the number of communities in the partition and its modularity, coverage, performance, density, and separability.

In [26]:
num_community = None # number of communities in the partition
mod = None   # modularity
cov = None   # coverage
perf = None  # performance
den = None # density
sep = None # separability

# YOUR CODE HERE
# Use the Clauset-Newman-Moore greedy modularity maximization algorithm
greedy_mod_communities = list(community.greedy_modularity_communities(G))
# Compute the number of communities
num_community = len(greedy_mod_communities)

# Compute the modularity
mod = quality.modularity(G, greedy_mod_communities)

# Compute the coverage
cov = quality.coverage(G, greedy_mod_communities)

# Compute the performance
perf = quality.performance(G, greedy_mod_communities)

# Compute the density
den = avg_measure(G, greedy_mod_communities, density_one_community)

# Compute the separability
sep = avg_measure(G, greedy_mod_communities, separability_one_community)
# raise NotImplementedError()

In [27]:
#hidden tests for Question 10 are within this cell

### Q11. (6 points, Autograded) Using [asynchronous Fluid Communities algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.asyn_fluid.asyn_fluidc.html#networkx.algorithms.community.asyn_fluid.asyn_fluidc) with parameters  `max_iter` = 100 and `seed` = 233 (see Tutorial for details), find the parameter `k` between `k`=3 and `k`=40 that maximizes the modularity of the partition. Note that `k` represents the number of communities in the partition. 

In [28]:
best_k = None #optimal value of k
fluid_communities = None #partition returned by the algorithm with k = best_k

from networkx.algorithms.community import asyn_fluidc, modularity
# Initialize best_modularity to a very low value
best_modularity =-1

# YOUR CODE HERE
# Iterate over possible k values from 3 to 40
for k in range(3, 41):
    # Find communities using Fluid Communities algorithm
    communities = list(asyn_fluidc(G, k, max_iter=100, seed=233))
    
    # Calculate modularity for current partition
    current_modularity = modularity(G, communities)
    
    # If current modularity is better than best so far, update best_modularity, best_k and fluid_communities
    if current_modularity > best_modularity:
        best_modularity = current_modularity
        best_k = k
        fluid_communities = communities
# raise NotImplementedError()

In [29]:
#hidden tests for Question 11 are within this cell

### Q12. (6 points, Autograded) Using [asynchronous Fluid Communities algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.asyn_fluid.asyn_fluidc.html#networkx.algorithms.community.asyn_fluid.asyn_fluidc) with parameters  `max_iter` = 100, `seed` = 233, and the value of `k` you found in the previous question, compute the number of communities in the partition and its modularity, coverage, performance, density, and separability.

In [30]:
num_community = None # number of communities in the partition
mod = None   # modularity
cov = None   # coverage
perf = None  # performance
den = None # density
sep = None # separability

# Number of communities in the partition
num_community = best_k

# Calculate modularity
mod = quality.modularity(G, fluid_communities)

# Calculate coverage
cov = quality.coverage(G, fluid_communities)

# Calculate performance
perf = quality.performance(G, fluid_communities)

# Calculate density
den = avg_measure(G, fluid_communities, density_one_community)

# Calculate separability
sep = avg_measure(G, fluid_communities, separability_one_community)

# YOUR CODE HERE
# raise NotImplementedError()

In [31]:
#hidden tests for Question 12 are within this cell

### Q13. (6 points, Autograded) Create and print a pandas DataFrame where each row represents a community detection algorithm and each column is a quality measure, as described by the following scheme:


|.|Method name|num_community|modularity|coverage|performance|density|separability|
|-|-|-|-|-|-|-|-|
|0|Girvan–Newman|
|1|Greedy modularity maximization|
|2|Fluid Communities|
|3|Label propogation|


In [32]:
compare = None # assign your pandas dataframe here
# result = {}
# num_community = len(max_mod_communities)
# modularity = quality.modularity(G, max_mod_communities)
# coverage = quality.coverage(G, max_mod_communities)
# performance = quality.performance(G, max_mod_communities)
# density = avg_measure(G, max_mod_communities, density_one_community)
# separability = avg_measure(G, max_mod_communities, separability_one_community)
# result['Girvan-Newman'] = [num_community,modularity,coverage,performance,density,separability]

# #####

# greedy_mod_communities = list(community.greedy_modularity_communities(G))
# num_community = len(greedy_mod_communities)
# modularity = quality.modularity(G, greedy_mod_communities)
# coverage = quality.coverage(G, greedy_mod_communities)
# performance = quality.performance(G, greedy_mod_communities)
# density = avg_measure(G, greedy_mod_communities, density_one_community)
# separability = avg_measure(G, greedy_mod_communities, separability_one_community)
# result['Greedy modularity maximization'] = [num_community,modularity,coverage,performance,density,separability]

# #####

# num_community = best_k
# modularity = quality.modularity(G, fluid_communities)
# coverage = quality.coverage(G, fluid_communities)
# performance = quality.performance(G, fluid_communities)
# density = avg_measure(G, fluid_communities, density_one_community)
# separability = avg_measure(G, fluid_communities, separability_one_community)
# result['Fluid Communities'] = [num_community,modularity,coverage,performance,density,separability]

# #####

# label_prop_communities = list(community.label_propagation_communities(G))
# num_community = len(label_prop_communities)
# modularity = quality.modularity(G, label_prop_communities)
# coverage = quality.coverage(G, label_prop_communities)
# performance = quality.performance(G, label_prop_communities)
# density = avg_measure(G, label_prop_communities, density_one_community)
# separability = avg_measure(G, label_prop_communities, separability_one_community)
# result['Label propogation'] = [num_community,modularity,coverage,performance,density,separability]

# compare = pd.DataFrame(result).T.reset_index().rename(columns = {'index':'Method name',0:'num_community'\
#                                                        ,1:'modularity',2:'coverage',3:'performance',4:'density'\
#                                                       ,5:'separability'})

algorithms = {
    'Girvan-Newman': max_mod_communities,
    'Greedy modularity maximization': greedy_mod_communities,
    'Fluid Communities': fluid_communities,
    'Label propogation': label_prop_communities
}

metrics = ['num_community', 'modularity', 'coverage', 'performance', 'density', 'separability']
result = {}

for algorithm, communities in algorithms.items():
    num_community = len(communities)
    modularity = quality.modularity(G, communities)
    coverage = quality.coverage(G, communities)
    performance = quality.performance(G, communities)
    density = avg_measure(G, communities, density_one_community)
    separability = avg_measure(G, communities, separability_one_community)
    result[algorithm] = [num_community, modularity, coverage, performance, density, separability]

compare = pd.DataFrame.from_dict(result, orient='index', columns=metrics).reset_index().rename(columns={'index': 'Method name'})

compare

,Method name,num_community,modularity,coverage,performance,density,separability
0,Girvan-Newman,35,0.580687,0.815252,0.888706,0.654218,1.079485
1,Greedy modularity maximization,14,0.550356,0.805892,0.775095,0.608520,1.480856
2,Fluid Communities,11,0.590856,0.728249,0.915128,0.211423,1.374656
3,Label propogation,28,0.584188,0.808808,0.857888,0.618999,1.285533


In [33]:
#hidden tests for Question 13 are within this cell

### Q14. (10 points, Manually graded) Does it appear that an algorithm consistently performs better than the others across the different quality measures? Explain. 

You do not need to explain your answer based on the details of the algorithms or quality measures. Do so only based on the quality scores on the dataframe in Q13. 

You should not consider the number of communities to answer this question. 



Based on the quality scores in the compare dataframe, it does not appear that one algorithm consistently performs better than the others across all the quality measures.

Looking at the scores for modularity, coverage, performance, density, and separability, we can see that different algorithms have higher values in different measures. For example, the Girvan-Newman algorithm has the highest coverage score, while the Fluid Communities algorithm has the highest modularity and performance scores. The Greedy modularity maximization algorithm has the highest separability score.

This indicates that each algorithm has its strengths and weaknesses in terms of different quality measures. The choice of the best algorithm may depend on the specific measure that is considered most important for the given analysis or application.

Therefore, based on the quality scores in the DataFrame, it does not seem that one algorithm consistently outperforms the others across all the quality measures.

In [34]:
science_domain

,num_community,modularity,coverage,performance,density,separability
science_domain_partitions,4.0,0.39848,0.715206,0.742542,0.069375,1.140328


### Q15. (10 points, Manually graded) Comparing the quality of the partition based on the science domain with that of the partitions generated by the various algorithms, from a network structure perspective, does the science domain appear to be a good way to partition the network into communities? Explain.

Recall that you already computed the quality of the partition based on science domain in Q6. Be sure to use those results when answering this question. 

You should not consider the number of communities to determine if a partition is "good." For this question, focus on the quality scores. 

Based on the data we observe that for different quality measures such as coverage, performance, density, and separability,the partitions generated by the algorithms consistently have higher scores compared to the science domain partition.

Therefore, science domain partition may not be the most effective way to partition the network into communities from a network structure perspective. The partitions generated by the algorithms consistently demonstrate higher quality scores across multiple measures compared to the science domain partition.

# End